# Baseline Method using LSTM

In [14]:
df = swapped_data

In [15]:
df.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox,ref_length,trn_length,tox_change
0,0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.785171,0.010309,0.981983,0.014195,95,96,0.967788
1,1,you're becoming disgusting.,Now you're getting nasty.,0.749687,0.071429,0.999039,0.065473,25,27,0.933567
2,2,"well, we can spare your life.","Well, we could spare your life, for one.",0.919051,0.268293,0.985068,0.213313,40,29,0.771755
3,3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.664333,0.309524,0.994215,0.053362,41,28,0.940853
4,4,I have orders to kill her.,I've got orders to put her down.,0.726639,0.181818,0.999348,0.009402,32,26,0.989946


In [16]:
from collections import Counter

# Combine all texts from the 'reference' and 'translation' columns
all_texts = swapped_data['reference'].tolist() + swapped_data['translation'].tolist()

# Tokenize the text
tokenized_texts = [word for text in all_texts for word in text.split()]

# Count the unique words
word_counts = Counter(tokenized_texts)
vocabulary_size = len(word_counts)

vocabulary_size



314702

In [17]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
# from tensorflow.keras.optimizers import Adam
# from sklearn.model_selection import train_test_split
# from nltk.translate.bleu_score import corpus_bleu

# data = df

# # Parameters
# max_vocab_size = 10000
# max_sequence_length = 100
# embedding_dim = 50

# # Text Tokenization
# tokenizer = Tokenizer(num_words=max_vocab_size, oov_token='<UNK>')
# tokenizer.fit_on_texts(data['reference'] + data['translation'])  # Fit on both reference and translation texts

# # Prepare input sequences
# input_sequences = tokenizer.texts_to_sequences(data['reference'])
# input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length)

# # Prepare output sequences
# output_sequences = tokenizer.texts_to_sequences(data['translation'])
# output_sequences = pad_sequences(output_sequences, maxlen=max_sequence_length)

# # Define the LSTM model
# input_ = Input(shape=(max_sequence_length,))
# x = Embedding(input_dim=max_vocab_size + 1, output_dim=embedding_dim)(input_)  # +1 for the OOV token
# x = LSTM(128, return_sequences=True)(x)
# output = Dense(max_vocab_size, activation='softmax')(x)  # Output layer size matches vocabulary size

# model = Model(input_, output)
# model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001))

# # Split the data into training and test sets
# input_train, input_test, output_train, output_test = train_test_split(
#     input_sequences, output_sequences, test_size=0.2, random_state=42)

# # Train the model
# model.fit(input_train, output_train, epochs=5)

In [18]:
# sample_text = "This is some shitty day"

# # Tokenize and pad the sample text
# sample_seq = tokenizer.texts_to_sequences([sample_text])
# sample_seq_padded = pad_sequences(sample_seq, maxlen=max_sequence_length)

# # Predict using the model
# predicted_output_seq = model.predict(sample_seq_padded)
# predicted_output_seq = predicted_output_seq.argmax(axis=-1)[0]  # Select the first (and only) sequence

# predicted_words = []
# for idx in predicted_output_seq:
#     word = tokenizer.index_word.get(idx)
#     if word is not None:
#         predicted_words.append(word)

# predicted_text = ' '.join(predicted_words)
# print("Sample Text:", sample_text)
# print("Predicted Text:", predicted_text)


In [19]:
!pip install tensorflow transformers

In [20]:
!pip install sentencepiece

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = df[['reference', 'translation']]

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(data, test_size=0.1)


train_df.to_csv('train_data.csv', index=False)
val_df.to_csv('val_data.csv', index=False)


In [23]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Load your data
train_df = pd.read_csv('train_data.csv')
val_df = pd.read_csv('val_data.csv')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['reference'], examples['translation'], padding='max_length', truncation=True)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
prefix = "detoxify:"
max_input_length = 256
max_target_length = 256

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["reference"]]
    targets = [ex for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [25]:
!pip install datasets

In [26]:
from datasets import Dataset, DatasetDict

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
ds = DatasetDict()
ds['train'] = train_ds
ds['val'] = val_ds
ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/519999 [00:00<?, ? examples/s]

Map:   0%|          | 0/57778 [00:00<?, ? examples/s]

In [27]:
hf_dataset_path = '../data/'
ds.save_to_disk(hf_dataset_path)

Saving the dataset (0/1 shards):   0%|          | 0/519999 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/57778 [00:00<?, ? examples/s]

In [28]:
!pip install accelerate -U

In [29]:
pip install torch

In [30]:
pip install transformers[torch]

In [39]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments
from transformers import T5Tokenizer, T5ForConditionalGeneration


model_name = 't5-base'
batch_size = 64
training_model_name = f"{model_name}-finetuned"

args = Seq2SeqTrainingArguments(
    output_dir=f'../models/{training_model_name}',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    # report_to='wandb',
)

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 1.8 MB/s eta 0:00:00


In [40]:
from datasets import load_metric


metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [36]:
pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


In [41]:
from transformers import Seq2SeqTrainer


trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=ds['train'],
    eval_dataset=ds['val'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored